In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import lightgbm as lgb

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")

In [5]:
def preprocess(df):
  df.index = pd.to_datetime(df["datetime"])
  df = df["2014-05-01":]
  df = df.reset_index(drop=True)
  
  def weather(x):
    if x.find("快晴"):
      x = 0
    elif x.find("晴れ"):
      x = 1
    elif x.find("曇"):
      x = 2
    elif x.find("薄曇"):
      x = 3
    elif x.find("雨"):
      x = 4
    else:
      x = 5
    return x

  df["days"] = df.index
  df["precipitation"] = df["precipitation"].apply(lambda x : -1 if x=="--" else x).astype(np.float64)
  df["fun"] = df["remarks"].apply(lambda x: 1 if x=="お楽しみメニュー" else 0)
  df["curry"] = df["name"].apply(lambda x : 1 if x.find("カレー")>=0 else 0)
  df["weather"] = df["weather"].apply(weather)
  #df["subuta"] = df["name"].apply(lambda x : 1 if x.find("酢豚")>=0 else 0)
  #df["sp"] = df["remarks"].apply(lambda x : 1 if x==("スペシャルメニュー")>=0 else 0)
  #df["kd"] = df["remarks"].apply(lambda x : 1 if x==("料理長")>=0 else 0)
  
  categorical_cols = ["precipitation", "weather", "days", "fun", "curry", "week", "name", "remarks", "event"]
  
  for cols in categorical_cols:
    if cols in df.columns:
      df[cols] = df[cols].astype("category")
  return df

In [6]:
df_train = preprocess(train)
test["y"] = 0
df_test = preprocess(test)
cols = ["datetime","precipitation","weather","days","fun","curry","y"]


In [7]:
# ５月１日～９月３０日
df_val = df_train[df_train["datetime"] >="2014-7-28"][cols] # 検証データ
df_tr = df_train[df_train["datetime"] <"2014-7-28" ][cols] # 学習データ

In [8]:
df_val_x = df_val.drop(["y","datetime"],axis=1) # 説明変数の設定
df_tr_x = df_tr.drop(["y","datetime"],axis=1)

df_val["y"] = df_val["y"].astype(int) #エラー修正
df_tr["y"] = df_tr["y"].astype(int) #エラー修正

df_val_y = df_val["y"] # 目的変数の設定
df_tr_y = df_tr["y"]

In [9]:
trains = lgb.Dataset(df_tr_x,df_tr_y)
valids = lgb.Dataset(df_val_x,df_val_y)

In [10]:
params = {
    "objective":"regression",
    "metric":"mae"
}

In [11]:
model = lgb.train(params,trains,valid_sets=valids,num_boost_round=1000,callbacks=[lgb.early_stopping(stopping_rounds=100)])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 5
[LightGBM] [Info] Start training from score 68.592593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements


In [12]:
pred = model.predict(df_test.drop("datetime",axis=1))

ValueError: train and valid dataset categorical_feature do not match.